In [ ]:
# pyspark 잘 작동하나 테스트
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
spark.sql('SELECT "Test" as c1').show()
spark.stop()

In [1]:
#hdfs 대신 단순 csv를 읽어와서 데이터 처리 후 csv를 저장.
#sample 용으로 standalone으로 실제 작업시 mesos모드로 변경 필요

# spark 모듈 추가
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
# 그외 모듈 추가
import pandas as pd
import numpy as np

In [3]:
# spark context 설정
conf = SparkConf().setAppName("paySparkApp")
sc = SparkContext(conf=conf)
# sparksession 생성
sparksession = SparkSession(sc)


In [4]:

# customer data 읽기
customer = sparksession.read.csv("data/origin/customer.csv", inferSchema=True, header=True)
customer.show(2)

# transaction data 읽기
transaction = sparksession.read.csv("data/origin/transaction.csv", inferSchema=True, header=True)
transaction.show(2)

# 등급테이블 만든 후 읽기 필요



+---+----+--------+-----------+
| id|name|birthday|nationality|
+---+----+--------+-----------+
|  1|null|    null|       null|
|  2|null|    null|       null|
+---+----+--------+-----------+
only showing top 2 rows

+---+-------------------+-----------+-------+
| id|   transaction_date|customer_id| amount|
+---+-------------------+-----------+-------+
|  1|2019-01-01 00:00:01|       3001| 543500|
|  2|2019-01-01 00:00:44|       3002|1633600|
+---+-------------------+-----------+-------+
only showing top 2 rows



In [13]:
#customer pandas 형태로 변환
new_customer=customer.toPandas()

In [15]:
new_customer.shape

(10000, 4)

In [16]:
new_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
id             10000 non-null int32
name           0 non-null object
birthday       0 non-null object
nationality    0 non-null object
dtypes: int32(1), object(3)
memory usage: 273.6+ KB


In [18]:
#transaction pandas 형태로 변환
new_transaction=transaction.toPandas()

In [19]:
new_transaction.shape

(120000, 4)

In [20]:
new_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 4 columns):
id                  120000 non-null int32
transaction_date    120000 non-null datetime64[ns]
customer_id         120000 non-null int32
amount              120000 non-null int32
dtypes: datetime64[ns](1), int32(3)
memory usage: 2.3 MB


In [32]:
#최종적으로 결과가 적재될 customer_risk_result 생성
customer_risk_result = pd.DataFrame(columns=['base_yymm','customer_id','risk_score','risk_grade'])
customer_risk_result

,base_yymm,customer_id,risk_score,risk_grade


In [33]:
#new_customer id값을 customer_risk_result에 적재
customer_risk_result['customer_id'] = new_customer['id']
customer_risk_result

,base_yymm,customer_id,risk_score,risk_grade
0,NaN,1,NaN,NaN
1,NaN,2,NaN,NaN
2,NaN,3,NaN,NaN
3,NaN,4,NaN,NaN
4,NaN,5,NaN,NaN
...,...,...,...,...
9995,NaN,9996,NaN,NaN
9996,NaN,9997,NaN,NaN
9997,NaN,9998,NaN,NaN
9998,NaN,9999,NaN,NaN


In [34]:
#이제 계산해서 넣어야하는데... 
import datetime, time
this_yymm = datetime.date.today()
base_yymm = this_yymm.isoformat()
print(base_yymm)

2020-01-05


In [36]:
#배치돌리는 시간 적재
customer_risk_result['base_yymm'] = base_yymm
customer_risk_result.head()

,base_yymm,customer_id,risk_score,risk_grade
0,2020-01-05,1,NaN,NaN
1,2020-01-05,2,NaN,NaN
2,2020-01-05,3,NaN,NaN
3,2020-01-05,4,NaN,NaN
4,2020-01-05,5,NaN,NaN


In [ ]:
#이제 계산해야하는데...


In [ ]:
#application 닫기
sc.stop()